# Handling🧑‍🚒🧯💨 🔥errors🔥 in Haskell

Exceptional values are... exceptional. And as such, they both shouldn't happen regularly and are treated differently.

Haskell is a compiled language, so we can subdivide all possible errors into:

- ✅ Compiletime errors

Compiletime errors are awesome! We LOVE compiletime errors. Becasue that means that our program had something wrong in it, and our compiler found it and reported it to us **BEFORE** it reached the end user. At the end of the day, the most important thing is that our users enjoy our software and don't encounter:

- ❌ Runtime errors

Runtime errors are the worst! Because these error happen while running the program, they can, and usually do, happen to end users. Users that then get angry and stop paying or give us a 1-star review or something like that.

Compared most other programming languages, Haskell is exceptionally good at avoiding runtime errors. Thanks in big part to its strong type system that is able to catch those errors at compiletime. Esentially, moving errors from being  runtime to being compiletime errors.

Although, this doesn't mean that we can forget about runtime errors. As we said in previous lessons, even if we where able to write perfect code (which, we cannot), we're still running it in the real world. Where computers run out of memory, files that should exist don't, internet connections fail, users do unimaginable things, etc.

So, today, we're going to learn:

# Outline

- Exceptions
- a

## There're always `Exception`s to the rule

When our program runs, it usually follows an execution flow based on what we think will happen. For example:

```haskell
main :: IO ()
main = do
  putStrLn "provide the distance traveled by the object (in [m]):"
  d <- readFile "aNumber.txt"
  putStrLn "provide the time it took for the object to travel the previous distance (in [s]:"
  t <- getLine
  let v = (read d :: Int) `div` read t
  putStrLn $ "The object's velocity is about: " ++ show v ++ " [m/s]"
  putStrLn "Thank you for using this program!"
```

In this program, we ask the user to provide two numbers (the distance traveled by an object, and the time it took) using IO actions. Then, we use the `read` function to parse them into `Int` values and apply the `div` function to get the velocity of the object. At the end, we print back the velocity and a thank-you message.

Because Haskell is lazy, we don't know exactly when everything is evaluated. For example, the compiler might choose to evaluate `read t` before `read d`. But we know that, if everything works, each function and action will retrun a value of an expected type and, on each situation, we provide the value to other functions and actions that can handle them.

Sadly, the real world is messy, and unexpected things can–and do–happen. When something unexpected happens, that's when we get an `Exception`. What's special about `Exception`s is that they halt the execution of your program. For example, here are a few (but not all) `Exception`s that our program could throw at us:

If there's no `aNumber.txt` file:
```haskell
*** Exception: aNumber.txt: openFile: does not exist (No such file or directory)
```

If we don't provide a numbers to `read`:
```haskell
*** Exception: Prelude.read: no parse
```

If we provide `0`(zero) as time:
```haskell
*** Exception: divide by zero
```

Currently, all these `Exception`s cause our program to halt and crash. But, of course, a robust program should not collapse if something unexpected happens. Lukilly, we have a couple of ways to handle these `Exception`s, one of them is to handle `Exception`s directly.

## I'm the  `Exception` cause I have `class` 😎

We're going to speedrun the subject so you have an overall idea of how it works, and then we're goint to go over how to handle common runtime errors.

Handling exceptions in Haskell is a big subject. You can go crazy with this system creating your own errors and using alternative libraries if you want to. In this lesson we're going to cover the basics so you can handle common errors.

Exceptions are just types that are instance of the `Exception` type class.

```haskell
class (Typeable e, Show e) => Exception e where
    toException   :: e -> SomeException
    fromException :: SomeException -> Maybe e
    displayException :: e -> String
```

The `Exception` type class is a subclass of the `Typeable` type class (because `Typeable` defines methods to identify types at runtime, when the exceptions happen), and the `Show` type classes (to print the exception to the screen in a readable way).

The `Exception` type class has only three methods:
And nd it has three methods:
- `toException`: That transforms a value of type `e` to a `SomeException`.
- `fromException`: That takes a value of type `SomeException` and maybe returns a value of type `e`. We're going to explore more about the `Maybe` type more at the end of this lesson.
- `displayException`: To render the exception value in a human-friendly manner (basically, show).

There're a few new things here, but don't stress out. We don't have to 


- How's `Typable` defined? And what are its methods?: We don't care. We're not going to use it 
- How does the `SomeException` data type looks like and how it works?
- What's that `Maybe e`?

I'll 

To create our own simple `Exception`, we just need create a data type representing the exception, and make it an instance of the `Exception` type class. For example, If we're parsing all the cases a String represents the color of traffic lights, we could do something like:

<div class="alert alert-block alert-info">
<b>Note:</b> We have to import <code>Control.Exception</code> which is the standard library module that contains code related to <code>Exceptions</code>. 
</div>

```haskell
import Control.Exception

data TrafficLightException = TrafficLightIsOff | WrongColor String deriving (Show)

instance Exception TrafficLightException
```

And that's it! We dont need to provide the definition of the instance's behaviours because there's no minimal definition. The compiler can infer the functions from the data type.

And if now we want a way for this exception to appear when it needs to, we have to throw it!

### `throw` all the `Exception`s you want. I'll `catch` them all!

To throw exceptions, we have a couple of options. We're going to go with the standard:

```haskell
throwIO :: Exception e => e -> IO a
```

As the name suggests `throwIO` throws an exception within the `IO` context. Why is it within the impure `IO` context? Because throwing exceptions is a side effect. It has the side effect of halting the program. Now, technically, you can throw exceptions from pure code using a function called only `throw` that is outside `IO`. But its highly recommended to use `throwIO` to have your pure code custom-exception-free:

```haskell
trafficLights :: String -> IO String
trafficLights color = case color of
  "Red"    -> return "Stop!"
  "Yellow" -> return "Wait!"
  "Green"  -> return "Go!"
  "Black"  -> throwIO TrafficLightIsOff
   _       -> throwIO . WrongColor $ color ++ " is not a valid color!"
```

If we apply this function to a `String`  different than "Red", "Yellow" or "Green", it will throw our cusom exception and halt the program:

```haskell
>> trafficLights "Black"
*** Exception: TrafficLightIsOff


>> trafficLights "Rainbow"
*** Exception: WrongColor "Rainbow is not a valid color!"
```

Now, if we just do this, the only thing we did is to create a new way for our program to crash. 🤦‍♂️ To make this a useful endeavor, we're going to catch and handle this exception so our program is able to recover from it and become more resilient! 💪

Let's say we're using the `trafficLights` function as part of a self-driving car's AI:

```haskell
dumbAICar :: IO ()
dumbAICar = do
  putStrLn "What color is the traffic light?"
  color <- getLine
  trafficLights color >>= putStrLn . ("I'll " ++)
```

Handling exceptions requires handling side-effects, so the only way to catch and handle exceptions is within inpure `IO`.

Now comes the key difference between exceptions and regular values. Exceptions live outside your program's usual execution. That means we are able to handle Exceptions and restart the program's regular execution after! When an exception is thrown, it starts rolling back through the stack, looking for a catch that is able to handle that Exception.

There are seveal ways to catch an Exception and recover from it. For example, using the `catch` function:

```haskell
catch :: Exception e  -- Type e is an instance of Exception
      => IO a         -- The computation to run
      -> (e -> IO a)  -- Handler to invoke if an exception is raised
      -> IO a         -- I always return a value of type IO a
```

Based on the , we can use `catch` to hanle our `TrafficLightException` like this:

```haskell
-- dumbAICar function with exception handling using `catch`
dumbAICar' :: IO ()
dumbAICar' = do
  putStrLn "What color is the traffic light?"
  color <- getLine
  response <- trafficLights color `catch` handler
  putStrLn $ "I'll " ++ response
  where
    handler :: TrafficLightException -> IO String
    handler tl = do
      putStrLn $ "WARNING: " ++ show tl
      case tl of
        TrafficLightIsOff -> return "Go!"
        WrongColor color -> return "Stop!"
```

Now, it doesn't matter if the String isn't a valid color. The `trafficLights` function will raise an exception, and if that exception is of type `TrafficLightException`, we'll be able to catch it and give it to our `hanlder` to recover from it.

If it's any other exception, the exception will keep propagating up the stack looking for a `catch` that is able to handle it. If it doesn't find any, the program halts.

And that's the end of our speedrun. There are a ton of things we didn't cover, but everythings revolves aroun this notion of throwing and catching exceptions. If you can forsee an exception and know what the progam should do in that case, you can implement a handler to recover from it.

And how can we know the usual exceptions we could encounter? Look no further than in the same `Control.Exception` module we've been using. In there, we have data types representing Exceptions that come with the language. Like the ones we saw at teh beginning of the lesson.

For example, if you're dealing with numeric operations, you can take a look at the `ArithException` data type:

```haskell
data ArithException
  = Overflow
  | Underflow
  | LossOfPrecision
  | DivideByZero
  | Denormal
  | RatioZeroDenominator 
```

### Disadvantages of using the `Exception` mechanism vs using regular values

- More impure code, which makes your program harder to predict

- Exceptions are complicated

- ??

### Advantages of using the `Exception` mechanism vs using regular values

- Avoids certain cases of code bloating when checking for errors using ADTs

- The exception-as-a-value (Maybe/Eitehr) doesn't copose well with functions that take the unwraped value (we're forced to use `case` or to pattern-match). Which generates a loss in modularity.

- Accidental scrictness: It's not unavoidable, but it becomes very easy to make our functions strict.

- Async exceptions have nothing to do with the expression that is being evaludated when the external event ocurs.

## Tradeoffs

Regular values | Exceptions |
| --- | --- |
Safety | conciceness |

## Using regular values to maintain the flow

a

## `Maybe` give me a value? 🙏

In Haskell, optional values are represented by the `Maybe` type:

```haskell
data Maybe a = Nothing | Just a
```

As you can see, it's a simple type. It has one nullary constructor (remember that nullary constructors have nothing to do with null values, they are called like that because they don't have parameters) and one constructor with a polymorphic value.

This will be the first instance that we're going to discuss the meaning of behind the code and how that changes how we think about it. 

Take a look at this other data type:

```haskell
data Box a = Empty | Has a
```

We've been working with the `Box a` data type for quite a few lessons so far. It represents a box that can either be empty or has some value.

Both the `Maybe a` and the `Box a` types are technically the same data type but with different names. What changes is how we think about them. The `Box` type represents a box that contains values and the `Maybe` type represents a value that might not be there.

It sounds silly, but this little change in how we, as programmers, think about the code, makes a lot of difference in the way we write code and the solutions and algorithms we come up when solving problems. I'm not asking you to have an epiphany or something like that. It's just that when you see something like this:

```haskell
someFunction :: Int -> String -> Maybe String
```

Yo read it as:

`someFunction` takes an `Int` and a `String` and `Maybe` returns a `String`.

And that's pretty much how read the type. Now, let's see it in practice. One common example is to show how we can safely divide by zero.

In your regular day-to-day math, dividing by zero doesn't make any mathematical sense. When programming, it's the same, so, if we divide by zero using the `div` function like this:

```haskell
55 `div` 0

*** Exception: divide by zero
```

Our program crashes instantly, like /insert joke/

One of the first things one could think to resolve the issue, is to create a function that handles this specific case in a different way, sort of like this:

```haskell
safeDiv :: Integral a => a -> a -> a
safeDiv x 0 = ????? -- 0? -1? 9999999999999? 
safeDiv x y = x `div` y
```

The `saveDivide` funciton takes two values of type `Int` and returns a value of the same type. In the first case, we patternmatch to handle the case when the second `Int` is zero, and the second case, we provide the function's body for any other situation. The second case is easy... we know that `y` is not zero, so we can use `div` with full confidence. But how should we handle the first case?

There's no value of type `Int` that we could use to return that correctly represents this situation.

Here's when the `Maybe` type shines! In the case when no value of the type we want makes sense, we can change our function to have the option of not returning a value of that type at all!!:

In [1]:
safeDiv :: Integral a => a -> a -> Maybe a
safeDiv _ 0 = Nothing
safeDiv x y = Just (x `div` y)


15 `safeDiv` 3
15 `safeDiv` 0

Just 5

Nothing

And how's this better than the program crashing? Because now we gained the hability to handle the case when we divide by zero in our program! For example, if this is a user input, we can ask the user to input another number:

```haskell
velocity :: Integral a => a -> a -> Maybe a
velocity distance time = distance `safeDiv` time

main :: IO ()
main = do
    putStrLn "provide the distance traveled by the object (in [m]):"
    d <- getLine
    putStrLn "provide the time it took for the object to travel the previous distance (in [s]:"
    t <- getLine
    case velocity (read d) (read t) of
        Just v  -> putStrLn $ "The object's velocity is about: " ++ show v  ++ " [m/s]"
        Nothing -> do
            putStrLn "\nThe time can't be zero!"
            main
```

By using this technique, we completely avoided a runtime error. It doesn't matter which numbers the user chooses, our program won't crash! 🙌 But wait... we have another possible runtime error! 😵 What if the user doesn't write valid numbers as inputs? What if the user writes letters or symbols instead of numbers?

In that case, the `read` function won't be able to parse the String to a number and... you guessed it... our program crashes. 🫣

Worry not! `Maybe a` data type to the rescue! In this case, we woud like a `read` function that has the option to fail. Luckily for us, a function like that already comes with our base libraries, and it's inside the `Text.Read` module:

In [14]:
import Text.Read (readMaybe)

-- readMaybe :: Read a => String -> Maybe a

readMaybe "57" :: Maybe Integer
readMaybe "B00!" :: Maybe Integer

Just 57

Nothing

By using `readMaybe` intead of `read`, we avoid **all runtime errors due to a bad user input**:

```haskell
import Text.Read (readMaybe)


velocity :: Maybe Int -> Maybe Int -> Maybe Int
velocity (Just dist) (Just time) = dist `safeDiv` time
velocity  _           _          = Nothing

 
main :: IO ()
main = do
  putStrLn "provide the distance traveled by the object (in [m]):"
  d <- getLine
  putStrLn "provide the time it took for the object to travel the previous distance (in [s]):"
  t <- getLine
  case velocity (readMaybe d) (readMaybe t) of
    Just v -> putStrLn $ "The object's velocity is: " ++ show v ++ " [m/s]"
    Nothing -> do
      putStrLn "\nWrong inputs! Please, provide valid numbers (time can't be zero)!"
      main
```

There are a lot of pre-defined funcitons that use the `Maybe a` type. Both in the standard libraries, and in libraries made by other develoeprs. When you encounter them, you can use them all in the same way. Pattern-match, handle both cases, and you're good to go!

Now that we understand how to use the `Maybe a` data type, and optional values more generally, let's talk about why should you use it:

- You can handle the absense of value: `Maybe a` provides a type-safe way to indicate the absence of a value, preventing many forms of runtime errors.

- Your code is more robust: Using `Maybe a` forces you to explicitly handle both the case when you have a value and when you don't, ensuring you handle all possibilities and write more robust code.

- Expressing uncertainty: Optional values allow the developer to express uncertainty in a way that the consumer of the function is aware and can handle, without the need to know how the body of the function looks like.

- Composability: You can compose optional values as we did in the previous examples. We provided the output of `readMaybe` as inputs to `velocity` that used `safeDiv` inside. If something went wrong at at any time, the `Nothing` value would propagate throught the function tree.

Now, the `Maybe a` data type is not the only data type in town we can use to handle possibly problematic values. One downside of using `Maybe a` data type is that, when we have several layers of optional values, we don't know what went wrong! In the previous code, for example. If we got a `Nothing` at the end, what failed? The parsing of the first value? The parsing of the second value? Or the division by zero? At runtime, we have no way to find out!

Enter the `Either a b` data type:

## Ok, you `Either` give me a value or a reason why you didn't!

The `Either` data type is also pretty simple:

```haskell
data Either a b = Left a | Right b
```

In general, the `Either` type represents values with two possibilities: a value of type `Either a b` is either `Left a` or `Right b`. You cannot have both. This is not necessarily related with error handling or optional values. Either can be used in many different scenarios to simply represent a binary choice.

In the context of error handling, you can think of `Either` as something that builds on top of the `Maybe` data type to provide a way to log what went wrong. 

The way we use `Either` to handle errors is not forced in code, but by convention. The community overall uses the `Left` constructor to hold an error value and the `Right` constructor to hold a correct value. Based on the mnemonic that "right" also means "correct".

So, now that we know about `Either`, let's modify `saveDiv` to let us know if we try to divide by zero:

In [2]:
safeDiv :: Integral a => a -> a -> Either String a
safeDiv _ 0 = Left "You can't divide by zero, you fool!!"
safeDiv x y = Right (x `div` y)


15 `safeDiv` 3
15 `safeDiv` 0

Right 5

Left "You can't divide by zero, you fool!!"

As you can see, we chose `String` as the type parameter for `Left` so we can leave a message to the user or developer. Although using a simple `String` to leave a message to the developer is the intuitive thing to try first, using values that we could later handle programmatically is even better. Something like:

```haskell
data PermissionLevel = Guest | User | Admin

data UIError = WrongInput String | WrongPermission PermissionLevel | UserDidNotLogIn

safeDiv :: Integral a => a -> a -> Either UIError a
```

And further down the code, we can use this extra information to do things programmatically. For example, if `Either` returns `Left UserDidNotLogIn`, we can redirect to the login page.

To keep it simple, we'll stick to `Strings` for today's examples. Now, let's transform the rest of the code:

```haskell
import Text.Read (readEither)


velocity :: Either String Int -> Either String Int -> Either String Int
velocity (Right dist) (Right time) = dist `safeDiv` time
velocity (Left _)     (Right _)    = Left "Wrong distance input! Please, provide valid numbers!"
velocity (Right _)    (Left _)     = Left "Wrong time input! Please, provide valid numbers!"
velocity  _            _           = Left "Both input wrong?! Come on!!"

main :: IO ()
main = do
  putStrLn "provide the distance traveled by the object (in [m]):"
  d <- getLine
  putStrLn "provide the time it took for the object to travel the prevous distance (in [s]):"
  t <- getLine
  case velocity (readEither d) (readEither t) of
    Right v -> putStrLn $ "The object's velocity is: " ++ show v ++ " [m/s]"
    Left e -> do
      putStrLn e
      main
```

As you can see, the `velocity` function now needs to explicitly handle more cases because we care about which input failed to parse, but the code complexity didn't increase that much overall.

On top of that, we see that we have the `readEither` function available in `Text.Read` aswell:

```haskell
readEither :: Read a => String -> Either String a
```

This function, as you might imagine, is the `Either` equivalent of the `readMaybe` funcion. And, in this case, we decided to ignore the default error string and substitute it with our own when defining the `velocity` function to make them more fitted to the context.

## `Maybe` I should use `Either` one or the other?

When to use `Maybe` and when to use `Either`?

---

###  When should I use ADT and when `Exceptions`?

- Strive to use programmable exceptions as much as posssible (Maybe, Either, etc.) to implement pure algorithms.

---

---

---

## Homework

- Make the student write all the Maybe and Either functions in the standard library?

- Modify the homework from the previous lesson to avoid as many runtimes errors as possible?